In [1]:
%matplotlib inline
from __future__ import division, print_function
import os, sys, json
from glob import glob
import numpy as np
import scipy
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt
import utils; reload(utils)
from utils import plots, get_batches, plot_confusion_matrix, get_data, load_array, save_array,split_at, copy_weights
from numpy.random import random, permutation
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom

import keras
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.models import Sequential
from keras.layers import Input
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.layers.normalization import BatchNormalization

import vgg16; reload(vgg16)
from vgg16 import Vgg16

Using TensorFlow backend.


In [2]:
DATA_HOME_DIR = os.getcwd() + '/data'
train_path = DATA_HOME_DIR + '/redux/sample/train/'
valid_path = DATA_HOME_DIR + '/redux/sample/valid/'
test_path  = DATA_HOME_DIR + '/redux/sample/test/'
results_path = DATA_HOME_DIR + '/redux/sample/results/'

models_path = DATA_HOME_DIR + '/redux/sample/models/'

if not os.path.exists(models_path): os.mkdir(models_path)
if not os.path.exists(results_path): os.mkdir(results_path)

In [3]:
batch_size = 64
train_batches = get_batches(train_path, shuffle=False, batch_size = batch_size)
valid_batches = get_batches(valid_path, shuffle=False, batch_size = batch_size)

Found 200 images belonging to 2 classes.
Found 100 images belonging to 2 classes.


In [4]:
def onehot(x): return np.array(OneHotEncoder().fit_transform(x.reshape(-1,1)).todense())

In [5]:
train_classes = train_batches.classes
valid_classes = valid_batches.classes
train_labels = onehot(train_classes)
valid_labels = onehot(valid_classes)

In [6]:
model =  Vgg16().model
layers = model.layers
last_conv_idx = [index for index,layer in enumerate(layers)
                    if type(layer) is Convolution2D][-1]
conv_layers = layers[:last_conv_idx+1]
conv_models =  Sequential(conv_layers)
fc_layers = layers[last_conv_idx + 1:]

In [ ]:
train_features = conv_models.predict_generator(train_batches, train_batches.nb_sample)
valid_features = conv_models.predict_generator(valid_batches, valid_batches.nb_sample)

In [ ]:
save_array(models_path + 'train_convlayer_features.bc', train_features)
save_array(models_path + 'valid_convlayer_features.bc', valid_features)
train_features = load_array(models_path + 'train_convlayer_features.bc')
valid_features = load_array(models_path + 'valid_convlayer_features.bc')

In [ ]:
train_data = get_data(train_path)
valid_data = get_data(valid_path)
save_array(models_path + 'train_data.bc', train_data)
save_array(models_path + 'valid_data.bc', valid_data)

In [ ]:
train_data = load_array(models_path + 'train_data.bc')
valid_data = load_array(models_path + 'valid_data.bc')

In [ ]:
model.pop()
model.pop()
last_layer_train_features = model.predict_generator(train_batches, train_batches.nb_sample)
last_layer_valid_features = model.predict_generator(valid_batches, valid_batches.nb_sample)
save_array(models_path + 'train_lastlayer_features.bc', last_layer_train_features)
save_array(models_path + 'valid_lastlayer_features.bc', last_layer_valid_features)

In [ ]:
last_layer_train_features = load_array(models_path + 'train_lastlayer_features.bc')
last_layer_valid_features = load_array(models_path + 'valid_lastlayer_features.bc')

In [ ]:
test_data = get_data(test_path)
save_array(models_path + 'test_data.bc', test_data)
test_data = load_array(models_path + 'test_data.bc')

In [ ]:
def get_conv_model(model):
    layers = model.layers
    last_conv_idx = [index for index, layer in enumerate(layers)
                        if type(layer) is Convolution2D][-1]
    conv_layers = layers[:last_conv_idx+1]
    conv_model = Sequential(conv_layers)
    fc_layers = layers[last_conv_idx+1:]
    return conv_model, fc_layers, last_conv_idx

In [ ]:
def get_fc_layers(p, in_shape):
    return [
        MaxPooling2D(input_shape=in_shape),
        Flatten(),
        Dense(4096, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(4096, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(2, activation='softmax')
    ]

In [ ]:
def get_lastlayer_layers():
    return [
        BatchNormalization(input_shape=(4096,)),
        Dropout(0.5),
        Dense(2, activation='softmax')
    ]

In [ ]:
def train_last_layer(i):
    ll_layers = get_lastlayer_layers()
    ll_model = Sequential(ll_layers)
    ll_model.compile(optimizer=Adam(), loss = 'categorical_crossentropy', metrics=['accuracy'])
    ll_model.optimizer.lr = 1e-5
    ll_model.fit(last_layer_train_features, train_labels, validation_data=(last_layer_valid_features, valid_labels), nb_epoch=12)
    ll_model.optimizer.lr = 1e-7
    ll_model.fit(last_layer_train_features, train_labels, validation_data=(last_layer_valid_features, valid_labels), nb_epoch=1)
    ll_model.save_weights(models_path+'last_layer_bn' + i + '.h5')
    
    vgg = Vgg16()
    model = vgg.model
    model.pop(); model.pop(); model.pop()
    for layer in model.layers: layer.trainable=False
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    
    lastlayer_layers = get_lastlayer_layers()
    for layer in lastlayer_layers: model.add(layer)
    for l1,l2 in zip(ll_model.layers, model.layers[-3:]):
        l2.set_weights(l1.get_weights())
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    model.save_weights(models_path + 'bn' + i + '.h5')
    return model

In [ ]:
def train_dense_layers(i, model):
    conv_model, fc_layers, last_conv_idx = get_conv_model(model)
    conv_shape = conv_model.output_shape[1:]
    fc_model = Sequential(get_fc_layers(0.5, conv_shape))
    for l1, l2 in zip(fc_model.layers, fc_layers):
        weights = l2.get_weights()
        l1.set_weights(weights)
    fc_model.compile(optimizer=Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
    fc_model.fit(train_features, train_labels, nb_epoch=2, batch_size=batch_size,
                valudation_data = (validation_features, validation_labels))
    generator = image.ImageDataGenerator(rotation_range=10, width_shift_range=0.05,
                                        width_zoom_range=0.05, zoom_range=0.05,
                                        channel_shift_range=10, height_shift_range=0.05,
                                        shear_range=0.05, horizontal_flip=True)
    train_batches = generator.flow(train_data, train_labels, batch_size=batch_size)
    valid_batches = image.ImageDataGenerator().flow(valid_data, valid_labels, shuffle=False,
                                                   batch_size=batch_size)
    
    for layer in conv_model.layers: layer.trainable=False
    for layer in get_fc_layers(0.5, conv_shape): conv_model.add(layer)
    for l1,l2 in zip(conv_model.layers[last_conv_idx+1:], fc_model.layers):
        l1.set_weights(l2.get_weights())
    
    conv_model.compile(optimizer=Adam(1e-5), loss='categorical_crossentropy',
                      metrics=['accuracy'])
    conv_model.save_weights(models_path+'no_dropout_bn'+i+'.h5')
    conv_model.fit_generator(train_batches, samples_per_epoch=train_batches.N, nb_epoch=1,
                            validation_data=valid_batches, nb_val_samples=valid_batches.N)
    for layer in conv_model.layers[16:]: layer.trainable=True
    conv_model.fit_generator(train_batches, samples_per_epoch=train_batches.N, nb_epoch=8,
                            validation_data=valid_batches, nb_val_samples=valid_batches.N)
    
    conv_model.optimizer.lr=1e-7
    conv_model.fit_generator(train_batches, samples_per_epoch=train_batches.N, nb_epoch=10,
                            validation_data=valid_batches, nb_val_samples=valid_batches.N)
    conv_model.save_weights(models_path + 'augementation' + i + '.h5')

In [ ]:
for i in range(5):
    i = str(i)
    model = train_last_layer(i)
    train_dense_layers(i, model)